In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(font_scale=1.5)
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_curve, RocCurveDisplay, roc_auc_score, precision_recall_curve, PrecisionRecallDisplay, average_precision_score
from lightgbm import LGBMClassifier
import optuna
from dataclasses import dataclass, field

In [2]:
%load_ext autoreload
%autoreload 2

There are 13,068 rows in the raw dataset, but this in itself is not very useful information - we really want to know how many individual neonates are in the data (some appear on multiple rows, corresponding to, for example, multiple blood culture tests). This number can be found using the unique identifier column (`Uid`):

In [3]:
data_filepath = './data/sepsis_updated_data_Feb21-Sep23.csv'
raw_df = pd.read_csv(data_filepath)
print('n rows:')
print(len(raw_df))
print('n unique ids:')
print(len(raw_df['Uid'].unique()))

n rows:
13068
n unique ids:
12392


Check start and end dates in the dataset:

In [4]:
raw_df['Datetimeadmission'] = pd.to_datetime(raw_df['Datetimeadmission'])
print('First admission:')
print(raw_df['Datetimeadmission'].min())
print('Last admission:')
print(raw_df['Datetimeadmission'].max())

First admission:
2021-02-02 12:10:50
Last admission:
2023-09-30 15:58:36


Find median birthweight:

In [5]:
raw_df[['Uid', 'Birthweight']].drop_duplicates()['Birthweight'].median()

2700.0

For case fatality rate, we assume that all those who died had the date and time of their death recorded in Neotree:

In [6]:
n_died = len(raw_df.loc[~pd.isna(raw_df['Datetimedeath']), 'Uid'].unique())
print('n died:', n_died)
n_total = len(raw_df['Uid'].unique())
print('case fatality rate:', round(n_died / n_total * 1000))

n died: 1963
case fatality rate: 158


How many had blood tests taken, and what were the results?:

In [7]:
print('n cases with test taken:', len(raw_df.loc[~pd.isna(raw_df['Neolab_finalbcresult']), 'Uid'].unique()))
rejected_ids = raw_df.loc[raw_df['Neolab_finalbcresult'].isin(['Contaminant', 'Rej']), 'Uid'].unique()
non_rejected_ids = raw_df.loc[raw_df['Neolab_finalbcresult'].isin(['Neg', 'NegP', 'Pos', 'PosP']), 'Uid'].unique()
print('n cases with no non-rejected tests:', len(np.setdiff1d(rejected_ids, non_rejected_ids)))

n cases with test taken: 3033
n cases with no non-rejected tests: 502


Load `datamanager` class to start preparing the raw data for analysis, first looking at the breakdown of the blood test results after removing the rejected rows:

In [309]:
from src.datamanager import DataManager

data_manager = DataManager(data_filepath)
columns_of_interest = ['Apgar1', 'Apgar5', 'Age', 'Gender',
       'Satsair', 'Typebirth', 'Romlength', 
       'Gestation', 'Birthweight', 'Temperature', 'Skin',
       'Dangersigns', 'Signsrd', 'Wob', 'Activity', 'Umbilicus', 'Colour',
       'Rr', 'Vomiting', 'Abdomen', 'Fontanelle', 'Hr']

How many cases are included in the analysis?

In [152]:
n_included = len(data_manager.df['Uid'].unique())
print(n_included)

11890


In [153]:
n_with_diagnosis_recorded = len(data_manager.df.loc[~pd.isna(data_manager.df['Diagdis1']), 'Uid'].unique())
n_with_eons_diagnosis = len(data_manager.df.loc[data_manager.df['eons_diagnosis'], 'Uid'].unique())
print('n_with_diagnosis_recorded:', n_with_diagnosis_recorded)
print('n_with_eons_diagnosis:', n_with_eons_diagnosis)
print(f'pct of diagnoses that were EONS: {n_with_eons_diagnosis / n_with_diagnosis_recorded * 100:.3f}')

n_with_diagnosis_recorded: 8624
n_with_eons_diagnosis: 99
pct of diagnoses that were EONS: 1.148


In [154]:
n_with_death_recorded = len(data_manager.df.loc[~pd.isna(data_manager.df['Causedeath']), 'Uid'].unique())
n_with_death_recorded_as_eons = len(data_manager.df.loc[data_manager.df['eons_cause_of_death'], 'Uid'].unique())
print('n_with_death_recorded:', n_with_death_recorded)
print('n_with_death_recorded_as_eons:', n_with_death_recorded_as_eons)
print(f'pct of causes of death that were recorded as EONS: {n_with_death_recorded_as_eons / n_with_death_recorded * 100:.3f}')

n_with_death_recorded: 1878
n_with_death_recorded_as_eons: 47
pct of causes of death that were recorded as EONS: 2.503


Finding out how many blood culture tests were positive or negative is a bit trickier than it feels like it should be, because some preliminary results are overturned by final results, in other cases blood culture test results are superceded by clinician diagnosis, etc, and the deduplication code is affected by which target variable we want to deduplicate for. To get the definitive breakdown, we create a new `DataManager` to deduplicate specifically for the `bc_positive` variable:

In [298]:
bc_result_data_manager = DataManager(data_filepath)
bc_result_data_manager.remove_duplicate_predictors(columns_of_interest, 'bc_positive')
bc_result_data_manager.df['bc_positive'].value_counts(dropna=False)

None     9359
False    2052
True      479
Name: bc_positive, dtype: int64

We then look at which rows in the main `data_manager` would be excluded due to poor quality age data:

In [328]:
def find_pos_results_excluded_due_to_age(row):
    if not pd.isna(row['Neolab_finalbcresult']) and row['bc_positive']:
        if pd.isna(row['age_at_test']):
            if row['eons_diagnosis'] or row['eons_cause_of_death']:
                return 'other route to pos'
            else:
                return 'pos_result_excluded_due_to_missing_age_data'
        elif row['age_at_test'] < 0 or row['age_at_test'] >= 72:
            if row['eons_diagnosis'] or row['eons_cause_of_death']:
                return 'other route to pos'
            else:
                return 'pos_result_excluded_due_to_bad_age_data'
    return 'other'
data_manager.df['pos_results_excluded'] = data_manager.df.apply(find_pos_results_excluded_due_to_age, axis=1)
data_manager.df.drop_duplicates(['Uid', 'pos_results_excluded'])['pos_results_excluded'].value_counts()

other                                          11709
pos_result_excluded_due_to_bad_age_data          248
pos_result_excluded_due_to_missing_age_data        9
other route to pos                                 5
Name: pos_results_excluded, dtype: int64

Having done this, we deduplicate rows on our main `data_manager` with our composite outcome variable:

In [329]:
data_manager.remove_duplicate_predictors(columns_of_interest, 'bc_positive_or_diagnosis_or_cause_of_death')

To count the missing values:

In [351]:
all_features = ['Apgar1', 'Apgar5', 'Apgar10', 'Age', 'Gender',
       'Bsmmol', 'Satsair', 'Typebirth', 'Romlength',
       'Gestation', 'Birthweight', 'Temperature', 'Skin',
       'Dangersigns', 'Signsrd', 'Wob', 'Activity', 'Umbilicus', 'Colour',
       'Rr', 'Vomiting', 'Abdomen', 'Fontanelle', 'Hr']
is_float = data_manager.df[all_features].dtypes == 'float64'

for index, value in data_manager.df[np.array(all_features)[is_float]].isna().sum().sort_values(ascending=False).items(): 
       print(f'{index:<12} | {value:>5} | {value / len(data_manager.df):.3%}')

Bsmmol       | 11639 | 97.889%
Apgar10      | 10123 | 85.139%
Apgar5       |   851 | 7.157%
Apgar1       |   812 | 6.829%
Age          |   269 | 2.262%
Satsair      |   238 | 2.002%
Rr           |   119 | 1.001%
Birthweight  |    35 | 0.294%
Hr           |    15 | 0.126%
Gestation    |     3 | 0.025%
Temperature  |     0 | 0.000%


In [352]:
for index, value in data_manager.df[np.array(all_features)[-is_float]].isna().sum().sort_values(ascending=False).items(): 
       print(f'{index:<12} | {value:>5} | {value / len(data_manager.df):.3%}')

Wob          |  6418 | 53.978%
Romlength    |  1362 | 11.455%
Skin         |    83 | 0.698%
Abdomen      |    53 | 0.446%
Colour       |    31 | 0.261%
Vomiting     |    31 | 0.261%
Umbilicus    |    29 | 0.244%
Typebirth    |     3 | 0.025%
Signsrd      |     3 | 0.025%
Activity     |     2 | 0.017%
Fontanelle   |     2 | 0.017%
Dangersigns  |     1 | 0.008%
Gender       |     0 | 0.000%


To count values in the composite outcome variable with missing continuous values removed:

In [356]:
X_train, X_test, y_train, y_test = data_manager.get_X_y(columns_of_interest, seed=2024, y_label='bc_positive_or_diagnosis_or_cause_of_death')
y = pd.concat([y_train, y_test])
print('Total number of rows:', len(y))
pd.Series(y).value_counts()

Total number of rows: 10420


False    10103
True       317
Name: bc_positive_or_diagnosis_or_cause_of_death, dtype: int64